In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns




In [2]:
df = pd.read_csv('Mental Health Dataset.csv')

# https://www.kaggle.com/datasets/bhavikjikadara/mental-health-dataset

# Alle Spalten anzeigen
pd.set_option('display.max_columns', len(df.columns))

In [3]:
df.head()

,Timestamp,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,2014-08-27 11:29:31,Female,United States,Corporate,NaN,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,2014-08-27 11:31:50,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,2014-08-27 11:32:39,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,2014-08-27 11:37:59,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,2014-08-27 11:43:36,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292364 entries, 0 to 292363
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Timestamp                292364 non-null  object
 1   Gender                   292364 non-null  object
 2   Country                  292364 non-null  object
 3   Occupation               292364 non-null  object
 4   self_employed            287162 non-null  object
 5   family_history           292364 non-null  object
 6   treatment                292364 non-null  object
 7   Days_Indoors             292364 non-null  object
 8   Growing_Stress           292364 non-null  object
 9   Changes_Habits           292364 non-null  object
 10  Mental_Health_History    292364 non-null  object
 11  Mood_Swings              292364 non-null  object
 12  Coping_Struggles         292364 non-null  object
 13  Work_Interest            292364 non-null  object
 14  Social_Weakness     

In [5]:
df.shape

(292364, 17)

In [6]:
print('Dieser Datensatz besitz ' + str(df.duplicated().sum()) + ' Duplikate.')
#duplicates = df[df.duplicated(keep=False)].sort_values(by=list(df.columns))
#duplicates

Dieser Datensatz besitz 363 Duplikate.


In [7]:
categoricalVar = [col for col in df if df[col].dtype == 'object']
print('Kategorische Variablen: ' + str(categoricalVar))

numericalVar = [col for col in df if df[col].dtype != 'object']
print('Numerische Variablen: '+ str(numericalVar))

Kategorische Variablen: ['Timestamp', 'Gender', 'Country', 'Occupation', 'self_employed', 'family_history', 'treatment', 'Days_Indoors', 'Growing_Stress', 'Changes_Habits', 'Mental_Health_History', 'Mood_Swings', 'Coping_Struggles', 'Work_Interest', 'Social_Weakness', 'mental_health_interview', 'care_options']
Numerische Variablen: []


In [8]:
df_prep = df

NaNs

In [9]:
df.isnull().sum()[df.isnull().sum() > 0]

self_employed    5202
dtype: int64

In [10]:
df_prep['self_employed'].fillna('unknown', inplace=True)

/var/folders/jf/k1y_jzxs5177jm_fgv__dnww0000gn/T/ipykernel_16284/341960323.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_prep['self_employed'].fillna('unknown', inplace=True)


Unwichtige Spalten löschen

In [11]:
notRelevantVar =['Timestamp', 'mental_health_interview']

df_prep.drop(notRelevantVar, axis = 1, inplace = True)

Werte ändern

In [12]:
# Anzahl der Uniquen Werte ausgeben lassen
df_prep.nunique()

Gender                    2
Country                  35
Occupation                5
self_employed             3
family_history            2
treatment                 2
Days_Indoors              5
Growing_Stress            3
Changes_Habits            3
Mental_Health_History     3
Mood_Swings               3
Coping_Struggles          2
Work_Interest             3
Social_Weakness           3
care_options              3
dtype: int64

In [13]:
# Funktion: Binäre Werte in 0/1 ändern
# Input: Datensatz
# Output: Datensatz mit den neuen Werte, Mapping Dictionary 
# Funktionsweise: 
    # 1. Erstellung eines Mapping Dictionarys, 
    # 2. Die einzelnen Spalten werden durchgegangen und bei jenen wo die anzahl der einzigartigen Werte gleich zwei ist, wird der erste Werte gleich 0 gesetzt und der zweite auf 1
    # 3. Die Zuordnung der Werte wird in dem Mapping Dictionary festgehalten
def binaer_mapping_function(df):
    mapping_dict = {}
    for col in df.columns:
        if len(df[col].unique()) == 2:
            mapping_dict[col] = {df[col].unique()[0]: 0, df[col].unique()[1]: 1}
            df[col] = df[col].map(mapping_dict[col])
    return df, mapping_dict


df_prep, mapping_dict = binaer_mapping_function(df_prep)
print(mapping_dict)


{'Gender': {'Female': 0, 'Male': 1}, 'family_history': {'No': 0, 'Yes': 1}, 'treatment': {'Yes': 0, 'No': 1}, 'Coping_Struggles': {'No': 0, 'Yes': 1}}


In [15]:
list_columns_moreThanTwoUniqueValues = df.columns[df.nunique() > 2].to_list()
list_columns_moreThanTwoUniqueValues

['Country',
 'Occupation',
 'self_employed',
 'Days_Indoors',
 'Growing_Stress',
 'Changes_Habits',
 'Mental_Health_History',
 'Mood_Swings',
 'Work_Interest',
 'Social_Weakness',
 'care_options']

In [16]:
# One-hot encoding for categorical variables
for var in list_columns_moreThanTwoUniqueValues:
    dummies = pd.get_dummies(df_prep[var], prefix=var, dtype=int)
    df_prep = pd.concat([df_prep, dummies], axis=1)
    df_prep.drop(var, axis=1, inplace=True)  

In [17]:
df_ml = df_prep
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292364 entries, 0 to 292363
Data columns (total 73 columns):
 #   Column                           Non-Null Count   Dtype
---  ------                           --------------   -----
 0   Gender                           292364 non-null  int64
 1   family_history                   292364 non-null  int64
 2   treatment                        292364 non-null  int64
 3   Coping_Struggles                 292364 non-null  int64
 4   Country_Australia                292364 non-null  int64
 5   Country_Belgium                  292364 non-null  int64
 6   Country_Bosnia and Herzegovina   292364 non-null  int64
 7   Country_Brazil                   292364 non-null  int64
 8   Country_Canada                   292364 non-null  int64
 9   Country_Colombia                 292364 non-null  int64
 10  Country_Costa Rica               292364 non-null  int64
 11  Country_Croatia                  292364 non-null  int64
 12  Country_Czech Republic        

## Daten in Traings und Testdaten teilen

In [18]:
df_ml['treatment'].value_counts()

treatment
0    147606
1    144758
Name: count, dtype: int64

In [19]:
y = df_ml['treatment']
X = df_ml.loc[:,df_ml.columns != 'treatment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

rf = RandomForestClassifier(criterion='entropy', n_estimators=500, random_state=33)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_rf)
precision = precision_score(y_test, y_pred_rf)
recall = recall_score(y_test, y_pred_rf)
f1 = f1_score(y_test, y_pred_rf)
roc_auc = roc_auc_score(y_test, y_pred_rf)

# Ergebnisse ausgeben
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)


Accuracy: 0.7399483522309442
Precision: 0.75964414158622
Recall: 0.6937731217370259
F1 Score: 0.7252159456431385
ROC AUC: 0.7394584728820492


In [33]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(objective="binary:logistic", eval_metric = 'auc', 
random_state=33, n_estimators=500, learning_rate=0.2)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_xgb)
precision = precision_score(y_test, y_pred_xgb)
recall = recall_score(y_test, y_pred_xgb)
f1 = f1_score(y_test, y_pred_xgb)
roc_auc = roc_auc_score(y_test, y_pred_xgb)

# Ergebnisse ausgeben
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)


Accuracy: 0.7586920459015272
Precision: 0.784942099796099
Recall: 0.7054247484700757
F1 Score: 0.7430621312550076
ROC AUC: 0.7581269258424829


In [34]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred_continuous = regressor.predict(X_test)


# Umwandlung in Binäre Klassifikation (>= 0 -> 1, <0 -> 0) ?????????????????
temp = 0.5
y_pred_linRegression_binary = (y_pred_continuous >= temp).astype(int)


accuracy = accuracy_score(y_test, y_pred_linRegression_binary)
precision = precision_score(y_test, y_pred_linRegression_binary)
recall = recall_score(y_test, y_pred_linRegression_binary)
f1 = f1_score(y_test, y_pred_linRegression_binary)
roc_auc = roc_auc_score(y_test, y_pred_linRegression_binary) # !!!!!!

# Ergebnisse ausgeben
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)

Accuracy: 0.7169804867203666
Precision: 0.7322098783966371
Recall: 0.6745150917954569
F1 Score: 0.7021793510536829
ROC AUC: 0.7165299655254781


In [22]:
from sklearn.svm import SVC

svc = SVC(kernel='poly', probability=True)  
svc.fit(X_train, y_train)
# y_pred_svm = svc.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred_svm)
# precision = precision_score(y_test, y_pred_svm)
# recall = recall_score(y_test, y_pred_svm)
# f1 = f1_score(y_test, y_pred_svm)
# roc_auc = roc_auc_score(y_test, y_pred_svm) # !!!!!!

# # Ergebnisse ausgeben
# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)
# print("ROC AUC:", roc_auc)


In [ ]:
# from sklearn.svm import SVC

# svc = SVC(kernel='poly')  # Du kannst den Kernel auch ändern, z.B. 'rbf', 'poly', etc.
# svc.fit(X_train, y_train)
# y_pred = svc.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f'Genauigkeit: {accuracy * 100:.2f}%')

# # plot_confusion_matrix_model(y_pred)
# # results(svc, 'SVM')

In [21]:
# import itertools
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
# import matplotlib.pyplot as plt

# def results(model, modelname):
#     # Predicting the test set results
#     y_pred = model.predict(X_test)
#     y_proba = model.predict_proba(X_test)[:, 1]  # Probabilities for ROC curve

#     # Evaluating the model
#     accuracy = accuracy_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred)
#     recall = recall_score(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred)
#     roc_auc = roc_auc_score(y_test, y_proba)

#     labelname= modelname + ' (area = %0.2f)' % roc_auc


#     # Plotting ROC curve
#     fpr, tpr, thresholds = roc_curve(y_test, y_proba)
#     plt.figure()
#     plt.plot(fpr, tpr, label=labelname)
#     plt.plot([0, 1], [0, 1], 'k--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver Operating Characteristic')
#     plt.legend(loc="lower right")
#     plt.show()

#     # Print the scores
#     print(f'Accuracy: {accuracy}')
#     print(f'Precision: {precision}')
#     print(f'Recall: {recall}')
#     print(f'F1 Score: {f1}')
#     print(f'ROC AUC: {roc_auc}')


# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     fmt = '.2f' if normalize else 'd'
#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, format(cm[i, j], fmt),
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")

#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.tight_layout()


# def plot_confusion_matrix_model(y_predicted_labels):
#     cnf_matrix = confusion_matrix(y_test, y_predicted_labels)
#     np.set_printoptions(precision=2)
#     cnf_matrix

#     fig, axes = plt.subplots(1, 2, figsize=(10, 5))

#     #Without Normalization
#     plt.sca(axes[0])  
#     plot_confusion_matrix(cnf_matrix, classes=[0, 1], title='Confusion matrix, without normalization')

#     # Mit Normalisierung
#     plt.sca(axes[1]) 
#     plot_confusion_matrix(cnf_matrix, classes=[0, 1], normalize=True, title='Normalized confusion matrix')

#     # Zeige die Figur
#     plt.tight_layout()
#     plt.show()



Random Forest 

In [22]:
# from sklearn.ensemble import RandomForestClassifier

# forest = RandomForestClassifier(criterion='entropy', n_estimators=500, random_state=33)
# forest.fit(X_train, y_train)
# y_predicted_labels = forest.predict(X_test)

# plot_confusion_matrix_model(y_predicted_labels)
# results(forest, 'Random Forest')

In [23]:
# from sklearn.model_selection import RandomizedSearchCV

# n_estimators = [100, 500, 1000]
# max_features = ['auto', 'sqrt']
# max_depth = [20, 50, 100]
# min_samples_split = [2, 5, 10]
# min_samples_leaf = [1, 3, 5]
# bootstrap = [True, False]

# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# print(random_grid)

# rf = RandomForestClassifier()
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 2, verbose=2, random_state=33, n_jobs = -1)
# rf_random.fit(X_train, y_train)

# rf_random.best_params_


In [24]:
# from sklearn.ensemble import RandomForestClassifier

# forest = RandomForestClassifier(
#     criterion='entropy', n_estimators=100, random_state=123, min_samples_split = 10, min_samples_leaf = 3, max_features = 'sqrt', max_depth = 20, bootstrap = True)
# forest.fit(X_train, y_train)
# y_predicted_labels = forest.predict(X_test)


# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
# import matplotlib.pyplot as plt

# # Predicting the test set results
# y_pred = forest.predict(X_test)
# y_proba = forest.predict_proba(X_test)[:, 1]  # Probabilities for ROC curve

# # Evaluating the model
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# roc_auc = roc_auc_score(y_test, y_proba)

# # Plotting ROC curve
# fpr, tpr, thresholds = roc_curve(y_test, y_proba)
# plt.figure()
# plt.plot(fpr, tpr, label='Random Forest (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc="lower right")
# plt.show()

# # Print the scores
# print(f'Accuracy: {accuracy}')
# print(f'Precision: {precision}')
# print(f'Recall: {recall}')
# print(f'F1 Score: {f1}')
# print(f'ROC AUC: {roc_auc}')


 XGboost

In [25]:
# import xgboost as xgb

# xgb_model = xgb.XGBClassifier(objective="binary:logistic", eval_metric = 'auc', 
# random_state=33, n_estimators=500, learning_rate=0.2)
# xgb_model.fit(X_train, y_train)

# y_pred_xgb = xgb_model.predict(X_test)

In [26]:
# #Confusion Matrix
# # Compute confusion matrix
# cnf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
# np.set_printoptions(precision=2)

# #With Normalization
# plt.figure()
# plot_confusion_matrix(cnf_matrix_xgb, classes= [0,1],
#                       title='Confusion matrix, without normalization')
# # With normalization
# plt.figure()
# plot_confusion_matrix(cnf_matrix_xgb, classes= [0,1], normalize=True,
#                       title='Normalized confusion matrix')

# plt.show()

In [27]:
# # Predicting the test set results

# y_proba_xgb = forest.predict_proba(X_test)[:, 1]  # Probabilities for ROC curve

# # Evaluating the model
# accuracy = accuracy_score(y_test, y_pred_xgb)
# precision = precision_score(y_test, y_pred_xgb)
# recall = recall_score(y_test, y_pred_xgb)
# f1 = f1_score(y_test, y_pred_xgb)
# roc_auc = roc_auc_score(y_test, y_proba_xgb)

# # Plotting ROC curve
# fpr, tpr, thresholds = roc_curve(y_test, y_proba_xgb)
# plt.figure()
# plt.plot(fpr, tpr, label='Random Forest (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc="lower right")
# plt.show()

# # Print the scores
# print(f'Accuracy: {accuracy}')
# print(f'Precision: {precision}')
# print(f'Recall: {recall}')
# print(f'F1 Score: {f1}')
# print(f'ROC AUC: {roc_auc}')

Linear Regression

In [28]:
# from sklearn.linear_model import LinearRegression

# regressor = LinearRegression()
# regressor.fit(X_train, y_train)
# y_pred_continuous = regressor.predict(X_test)


# # Umwandlung in Binäre Klassifikation (>= 0 -> 1, <0 -> 0) ?????????????????
# temp = 0.5
# y_pred_binary = (y_pred_continuous >= temp).astype(int)


# accuracy = accuracy_score(y_test, y_pred_binary)
# precision = precision_score(y_test, y_pred_binary)
# recall = recall_score(y_test, y_pred_binary)
# f1 = f1_score(y_test, y_pred_binary)
# roc_auc = roc_auc_score(y_test, y_pred_binary) # !!!!!!

# # Ergebnisse ausgeben
# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)
# print("ROC AUC:", roc_auc)

Support Vector Machine (SVM)

In [29]:
# from sklearn.svm import SVC

# svc = SVC(kernel='poly')  # Du kannst den Kernel auch ändern, z.B. 'rbf', 'poly', etc.
# svc.fit(X_train, y_train)
# y_pred = svc.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f'Genauigkeit: {accuracy * 100:.2f}%')

# # plot_confusion_matrix_model(y_pred)
# # results(svc, 'SVM')

In [30]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score

# # Datensatz erstellen
# data = {
#     'Alter': [25, 45, 35, 50, 23, 39, 28, 41, 30, 52, 27, 33],
#     'Einkommen': [30, 50, 60, 40, 25, 55, 45, 35, 50, 65, 40, 55],
#     'Anzahl_Besuche': [5, 2, 3, 4, 6, 1, 3, 2, 4, 5, 1, 3],
#     'Ausgaben': [500, 200, 300, 400, 600, 100, 250, 150, 350, 450, 80, 300],
#     'Zufriedenheit': [4, 3, 5, 2, 5, 3, 4, 2, 4, 3, 1, 5],
#     'Kaufentscheidung': [1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1]
# }

# # In DataFrame umwandeln
# df = pd.DataFrame(data)

# # Merkmale und Zielvariable definieren
# X = df[['Alter', 'Einkommen', 'Anzahl_Besuche', 'Ausgaben', 'Zufriedenheit']]
# y = df['Kaufentscheidung']

# # Datensatz in Trainings- und Testdaten aufteilen
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # SVM Modell erstellen und trainieren
# svm_model = SVC(kernel='linear')
# svm_model.fit(X_train, y_train)

# # Vorhersagen auf den Testdaten
# y_pred = svm_model.predict(X_test)

# # Genauigkeit berechnen
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Genauigkeit: {accuracy * 100:.2f}%')


Ensemble Methode

In [31]:
# from sklearn.ensemble import VotingClassifier

# # Annahme: Sie haben bereits trainierte Modelle rf und xgb

# # Erstellen des Ensembles
# ensemble = VotingClassifier(estimators=[
#     ('random_forest', forest),
#     ('xgboost', xgb_model)
# ], voting='soft')  # 'soft' Voting für Wahrscheinlichkeiten

# # Ensemble trainieren
# ensemble.fit(X_train, y_train)

# # Vorhersagen machen
# predictions = ensemble.predict(X_test)

# accuracy = accuracy_score(y_test, predictions)
# precision = precision_score(y_test, predictions)
# recall = recall_score(y_test, predictions)
# f1 = f1_score(y_test, predictions)
# roc_auc = roc_auc_score(y_test, predictions)

# # Ergebnisse ausgeben
# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)
# print("ROC AUC:", roc_auc)